# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [278]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Ask for parameters

In [279]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [280]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Binary Variables (Activation):**

*  `x_AB`, `x_AC`, `x_AF`, `x_BC`, `x_BD`, `x_BE`, `x_CE`, `x_CF`, `x_DE`, `x_EG`, `x_FB`, `x_FG` 
    * Each variable represents a pipe. 
    * Value 1: Pipe is activated.
    * Value 0: Pipe is deactivated.

**Continuous Variables (Water Flow):**

* `f_in(t)`: Water entering the system at point A at time step `t` (calculated from the given function).
* `f_out(t)`: Water exiting the system at point G at time step `t`. 
* `f_AB(t)`, `f_AC(t)`, `f_AF(t)`, `f_BC(t)`, `f_BD(t)`, `f_BE(t)`, `f_CE(t)`, `f_CF(t)`, `f_DE(t)`, `f_EG(t)`, `f_FB(t)`, `f_FG(t)`
    * Each variable represents a pipe.
    * Represents the water flow through the respective pipe at time step `t`.

**Continuous Variables (Water Released):**

* `r_B(t)`, `r_C(t)`, `r_D(t)`, `r_E(t)`, `r_F(t)`
    * Each variable represents a connection point (B, C, D, E, F).
    * Represents the amount of water released at the respective point at time step `t`.

**Sets:**

*  `T = {1, 2, ..., 10}` (Set of time steps) 


# 2. Ask for objective

In [281]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [282]:
Markdown(response2.text)

Maximize: 
  $$\sum_{t=1}^{10} f_{out}(t)$$ 
 
This objective function directly represents the goal of maximizing the total water flow exiting the system at point G over the 10 time steps. 


# 3. Ask for constraints

In [283]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [284]:
Markdown(response3.text)

## Constraints:

**1. Pipe Activation:**

*  Only 10 pipes can be activated: 
   `x_AB + x_AC + x_AF + x_BC + x_BD + x_BE + x_CE + x_CF + x_DE + x_EG + x_FB + x_FG = 10`

**2. Flow Capacity:**

*  Flow through a pipe cannot exceed its capacity:
    * `0 ≤ f_AB(t) ≤ 3x_AB` for all `t` in `T`
    * `0 ≤ f_AC(t) ≤ 6x_AC` for all `t` in `T`
    * ... (similarly for all other pipes)

**3. Flow Conservation at Each Node:**

* **Node A:**
    * `f_in(t) = f_AB(t) + f_AC(t) + f_AF(t)` for all `t` in `T`
* **Node B:**
    * `f_AB(t) + f_FB(t) = f_BC(t) + f_BD(t) + f_BE(t) + r_B(t)` for all `t` in `T`
* **Node C:**
    * `f_AC(t) + f_BC(t) = f_CE(t) + f_CF(t) + r_C(t)` for all `t` in `T`
* **Node D:**
    * `f_BD(t) + f_DE(t) = r_D(t)` for all `t` in `T` 
* **Node E:** 
    * `f_BE(t) + f_CE(t) = f_DE(t) + f_EG(t) + r_E(t)` for all `t` in `T`
* **Node F:**
    * `f_AF(t) + f_CF(t) = f_FB(t) + f_FG(t) + r_F(t)` for all `t` in `T`
* **Node G:**
    * `f_EG(t) + f_FG(t) = f_out(t)` for all `t` in `T`

**4. Non-negativity of Flow and Release:**

* All flow and release variables must be non-negative:
    * `f_ij(t) ≥ 0` for all pipes `ij` and all `t` in `T`
    * `r_k(t) ≥ 0` for all nodes `k` (B, C, D, E, F) and all `t` in `T` 


## 3. Generate the pyomo code

In [285]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [286]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data for f_in(t) - Replace with actual function or data
def f_in_func(t):
    # Example: Sinusoidal inflow
    return 5 * pyo.sin(t * 3.14159 / 10) + 10 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10)  # Time steps

# Variables
# Activation (Binary)
model.x_AB = pyo.Var(within=pyo.Binary)
model.x_AC = pyo.Var(within=pyo.Binary)
model.x_AF = pyo.Var(within=pyo.Binary)
model.x_BC = pyo.Var(within=pyo.Binary)
model.x_BD = pyo.Var(within=pyo.Binary)
model.x_BE = pyo.Var(within=pyo.Binary)
model.x_CE = pyo.Var(within=pyo.Binary)
model.x_CF = pyo.Var(within=pyo.Binary)
model.x_DE = pyo.Var(within=pyo.Binary)
model.x_EG = pyo.Var(within=pyo.Binary)
model.x_FB = pyo.Var(within=pyo.Binary)
model.x_FG = pyo.Var(within=pyo.Binary)

# Flow (Continuous)
model.f_in = pyo.Var(model.T)
model.f_out = pyo.Var(model.T)
model.f_AB = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_AC = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_AF = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_BC = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_BD = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_BE = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_CE = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_CF = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_DE = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_EG = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_FB = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_FG = pyo.Var(model.T, within=pyo.NonNegativeReals)

# Release (Continuous)
model.r_B = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_C = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_D = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_E = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_F = pyo.Var(model.T, within=pyo.NonNegativeReals)

# Constraints

# 1. Pipe Activation
def pipe_activation_rule(model):
    return (model.x_AB + model.x_AC + model.x_AF + model.x_BC + model.x_BD + 
            model.x_BE + model.x_CE + model.x_CF + model.x_DE + model.x_EG + 
            model.x_FB + model.x_FG == 10)
model.pipe_activation_constraint = pyo.Constraint(rule=pipe_activation_rule)

# 2. Flow Capacity
def flow_capacity_rule(model, t, pipe, x, capacity):
    return (0, model.getattr(f"f_{pipe}")[t], capacity * model.getattr(f"x_{pipe}")) 
model.flow_capacity_AB = pyo.Constraint(model.T, rule=lambda model, t: flow_capacity_rule(model, t, "AB", model.x_AB, 3))
model.flow_capacity_AC = pyo.Constraint(model.T, rule=lambda model, t: flow_capacity_rule(model, t, "AC", model.x_AC, 6))
# ... (similarly for all other pipes)

# 3. Flow Conservation at Each Node
def node_A_rule(model, t):
    return model.f_in[t] == model.f_AB[t] + model.f_AC[t] + model.f_AF[t]
model.node_A_constraint = pyo.Constraint(model.T, rule=node_A_rule)

def node_B_rule(model, t):
    return (model.f_AB[t] + model.f_FB[t] == 
            model.f_BC[t] + model.f_BD[t] + model.f_BE[t] + model.r_B[t])
model.node_B_constraint = pyo.Constraint(model.T, rule=node_B_rule)

# ... (similarly for nodes C, D, E, F, G) 

# Objective Function
def objective_rule(model):
    return pyo.summation(model.f_out[t] for t in model.T)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Set inflow values using the sample data function
for t in model.T:
    model.f_in[t] = f_in_func(t)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results or access solution information
print(results)
print("Total outflow:", pyo.value(model.objective)) 
```

**Explanation:**

1. **Import Pyomo:** `import pyomo.environ as pyo` imports the Pyomo library.
2. **Sample Data:** I've used a sample sinusoidal function for `f_in(t)`. You'll need to replace this with your actual inflow data or function.
3. **Model Initialization:** `model = pyo.ConcreteModel()` creates a concrete Pyomo model.
4. **Sets:** `model.T = pyo.RangeSet(1, 10)` defines the set of time steps.
5. **Variables:**
   - **Binary:** `pyo.Var(within=pyo.Binary)` is used to define binary variables (`x_AB`, `x_AC`, etc.).
   - **Continuous:** `pyo.Var(model.T, within=pyo.NonNegativeReals)` is used for flow and release variables, which are continuous and non-negative. 
6. **Constraints:**  Each constraint is defined using a rule function. The `pyo.Constraint()` function applies these rules over the appropriate sets.
7. **Objective Function:** The `objective_rule` function defines the objective to maximize, and `pyo.Objective()` creates the objective object.
8. **Setting Inflow:** The loop `for t in model.T:` sets the values of `f_in[t]` based on the sample data function.
9. **Solving:** A solver is selected (`cbc` in this case) and used to solve the model.
10. **Results:** The results are printed, including the total outflow.

Remember that this code uses sample data for the inflow (`f_in(t)`). You need to replace the sample function with your actual data or function to obtain meaningful results. Also, you'll likely need to install a suitable solver (like CBC, GLPK, or others) if you haven't already. 


## 4. Input problem data and try running the generated code

In [287]:
import pyomo.environ as pyo

# Sample data for f_in(t) - Replace with actual function or data
def f_in_func(t):
    # Example: Sinusoidal inflow
    return 5 * pyo.sin(t * 3.14159 / 10) + 10 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10)  # Time steps

# Variables
# Activation (Binary)
model.x_AB = pyo.Var(within=pyo.Binary)
model.x_AC = pyo.Var(within=pyo.Binary)
model.x_AF = pyo.Var(within=pyo.Binary)
model.x_BC = pyo.Var(within=pyo.Binary)
model.x_BD = pyo.Var(within=pyo.Binary)
model.x_BE = pyo.Var(within=pyo.Binary)
model.x_CE = pyo.Var(within=pyo.Binary)
model.x_CF = pyo.Var(within=pyo.Binary)
model.x_DE = pyo.Var(within=pyo.Binary)
model.x_EG = pyo.Var(within=pyo.Binary)
model.x_FB = pyo.Var(within=pyo.Binary)
model.x_FG = pyo.Var(within=pyo.Binary)

# Flow (Continuous)
model.f_in = pyo.Var(model.T)
model.f_out = pyo.Var(model.T)
model.f_AB = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_AC = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_AF = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_BC = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_BD = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_BE = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_CE = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_CF = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_DE = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_EG = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_FB = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_FG = pyo.Var(model.T, within=pyo.NonNegativeReals)

# Release (Continuous)
model.r_B = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_C = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_D = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_E = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.r_F = pyo.Var(model.T, within=pyo.NonNegativeReals)

# Constraints

# 1. Pipe Activation
def pipe_activation_rule(model):
    return (model.x_AB + model.x_AC + model.x_AF + model.x_BC + model.x_BD + 
            model.x_BE + model.x_CE + model.x_CF + model.x_DE + model.x_EG + 
            model.x_FB + model.x_FG == 10)
model.pipe_activation_constraint = pyo.Constraint(rule=pipe_activation_rule)

# 2. Flow Capacity
def flow_capacity_rule(model, t, pipe, x, capacity):
    return (0, model.getattr(f"f_{pipe}")[t], capacity * model.getattr(f"x_{pipe}")) 
model.flow_capacity_AB = pyo.Constraint(model.T, rule=lambda model, t: flow_capacity_rule(model, t, "AB", model.x_AB, 3))
model.flow_capacity_AC = pyo.Constraint(model.T, rule=lambda model, t: flow_capacity_rule(model, t, "AC", model.x_AC, 6))
# ... (similarly for all other pipes)

# 3. Flow Conservation at Each Node
def node_A_rule(model, t):
    return model.f_in[t] == model.f_AB[t] + model.f_AC[t] + model.f_AF[t]
model.node_A_constraint = pyo.Constraint(model.T, rule=node_A_rule)

def node_B_rule(model, t):
    return (model.f_AB[t] + model.f_FB[t] == 
            model.f_BC[t] + model.f_BD[t] + model.f_BE[t] + model.r_B[t])
model.node_B_constraint = pyo.Constraint(model.T, rule=node_B_rule)

# ... (similarly for nodes C, D, E, F, G) 

# Objective Function
def objective_rule(model):
    return pyo.summation(model.f_out[t] for t in model.T)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Set inflow values using the sample data function
for t in model.T:
    model.f_in[t] = f_in_func(t)

# Solve the model
solver = pyo.SolverFactory('cbc')  # Choose a suitable solver
results = solver.solve(model)

# Print results or access solution information
print(results)
print("Total outflow:", pyo.value(model.objective)) 


ERROR: Rule failed when generating expression for Constraint flow_capacity_AB
with index 1: AttributeError: 'ConcreteModel' object has no attribute
'getattr'
ERROR: Constructing component 'flow_capacity_AB' from data=None failed:
        AttributeError: 'ConcreteModel' object has no attribute 'getattr'


AttributeError: 'ConcreteModel' object has no attribute 'getattr'

## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [288]:
print(response.text)

## Variables:

**Binary Variables (Activation):**

*  `x_AB`, `x_AC`, `x_AF`, `x_BC`, `x_BD`, `x_BE`, `x_CE`, `x_CF`, `x_DE`, `x_EG`, `x_FB`, `x_FG` 
    * Each variable represents a pipe. 
    * Value 1: Pipe is activated.
    * Value 0: Pipe is deactivated.

**Continuous Variables (Water Flow):**

* `f_in(t)`: Water entering the system at point A at time step `t` (calculated from the given function).
* `f_out(t)`: Water exiting the system at point G at time step `t`. 
* `f_AB(t)`, `f_AC(t)`, `f_AF(t)`, `f_BC(t)`, `f_BD(t)`, `f_BE(t)`, `f_CE(t)`, `f_CF(t)`, `f_DE(t)`, `f_EG(t)`, `f_FB(t)`, `f_FG(t)`
    * Each variable represents a pipe.
    * Represents the water flow through the respective pipe at time step `t`.

**Continuous Variables (Water Released):**

* `r_B(t)`, `r_C(t)`, `r_D(t)`, `r_E(t)`, `r_F(t)`
    * Each variable represents a connection point (B, C, D, E, F).
    * Represents the amount of water released at the respective point at time step `t`.

**Sets:**

*  `T = {1, 

In [289]:
print(response2.text)

Maximize: 
  $$\sum_{t=1}^{10} f_{out}(t)$$ 
 
This objective function directly represents the goal of maximizing the total water flow exiting the system at point G over the 10 time steps. 



In [290]:
print(response3.text)

## Constraints:

**1. Pipe Activation:**

*  Only 10 pipes can be activated: 
   `x_AB + x_AC + x_AF + x_BC + x_BD + x_BE + x_CE + x_CF + x_DE + x_EG + x_FB + x_FG = 10`

**2. Flow Capacity:**

*  Flow through a pipe cannot exceed its capacity:
    * `0 ≤ f_AB(t) ≤ 3x_AB` for all `t` in `T`
    * `0 ≤ f_AC(t) ≤ 6x_AC` for all `t` in `T`
    * ... (similarly for all other pipes)

**3. Flow Conservation at Each Node:**

* **Node A:**
    * `f_in(t) = f_AB(t) + f_AC(t) + f_AF(t)` for all `t` in `T`
* **Node B:**
    * `f_AB(t) + f_FB(t) = f_BC(t) + f_BD(t) + f_BE(t) + r_B(t)` for all `t` in `T`
* **Node C:**
    * `f_AC(t) + f_BC(t) = f_CE(t) + f_CF(t) + r_C(t)` for all `t` in `T`
* **Node D:**
    * `f_BD(t) + f_DE(t) = r_D(t)` for all `t` in `T` 
* **Node E:** 
    * `f_BE(t) + f_CE(t) = f_DE(t) + f_EG(t) + r_E(t)` for all `t` in `T`
* **Node F:**
    * `f_AF(t) + f_CF(t) = f_FB(t) + f_FG(t) + r_F(t)` for all `t` in `T`
* **Node G:**
    * `f_EG(t) + f_FG(t) = f_out(t)` for all `t` in `

In [291]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data for f_in(t) - Replace with actual function or data
def f_in_func(t):
    # Example: Sinusoidal inflow
    return 5 * pyo.sin(t * 3.14159 / 10) + 10 

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 10)  # Time steps

# Variables
# Activation (Binary)
model.x_AB = pyo.Var(within=pyo.Binary)
model.x_AC = pyo.Var(within=pyo.Binary)
model.x_AF = pyo.Var(within=pyo.Binary)
model.x_BC = pyo.Var(within=pyo.Binary)
model.x_BD = pyo.Var(within=pyo.Binary)
model.x_BE = pyo.Var(within=pyo.Binary)
model.x_CE = pyo.Var(within=pyo.Binary)
model.x_CF = pyo.Var(within=pyo.Binary)
model.x_DE = pyo.Var(within=pyo.Binary)
model.x_EG = pyo.Var(within=pyo.Binary)
model.x_FB = pyo.Var(within=pyo.Binary)
model.x_FG = pyo.Var(within=pyo.Binary)

# Flow (Continuous)
model.f_in = pyo.Var(model.T)
model.f_out = pyo.Var(model.T)
model.f_AB = pyo.Var(model.T, within=pyo.NonNegativeReals)
model.f_AC = pyo.Var(model.T, within=pyo.NonNegativeR